# Monte Carlo approaches to prediction and control

In this notebook, you will implement the Monte Carlo approaches to prediction and control described in [Sutton and Barto's book, Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html). We will use the grid ```World``` class from the previous lecture, but now without relying on knowledge of the task dynamics, that is, without relying on knowledge about transition probabilities.

### Install dependencies

In [1]:
! pip install numpy pandas

### Imports

In [2]:
import numpy as np
import random
import sys          # We use sys to get the max value of a float
import pandas as pd # We only use pandas for displaying tables nicely
pd.options.display.float_format = '{:,.3f}'.format

### ```World``` class and globals

The ```World``` is a grid represented as a two-dimensional array of characters where each character can represent free space, an obstacle, or a terminal. Each non-obstacle cell is associated with a reward that an agent gets for moving to that cell (can be 0). The size of the world is _width_ $\times$ _height_ characters.

A _state_ is a tuple $(x,y)$.

An empty world is created in the ```__init__``` method. Obstacles, rewards and terminals can then be added with ```add_obstacle``` and ```add_reward```.

To calculate the next state of an agent (that is, an agent is in some state $s = (x,y)$ and performs and action, $a$), ```get_next_state()```should be called.

__Note that ```get_state_transition_probabilities``` has been removed and an agent must now rely on experience interacting with a world to learn.__

In [3]:
# Globals:
ACTIONS = ("up", "down", "left", "right")

# Rewards, terminals and obstacles are characters:
REWARDS = {" ": 0, ".": 0.1, "+": 10, "-": -10}
TERMINALS = ("+", "-") # Note a terminal should also have a reward assigned
OBSTACLES = ("#")

# Discount factor
gamma = 1

# The probability of a random move:
rand_move_probability = 0

class World:
  def __init__(self, width, height):
    self.width = width
    self.height = height
    # Create an empty world where the agent can move to all cells
    self.grid = np.full((width, height), ' ', dtype='U1')

  def add_obstacle(self, start_x, start_y, end_x=None, end_y=None):
    """
    Create an obstacle in either a single cell or rectangle.
    """
    if end_x == None: end_x = start_x
    if end_y == None: end_y = start_y

    self.grid[start_x:end_x + 1, start_y:end_y + 1] = OBSTACLES[0]

  def add_reward(self, x, y, reward):
    assert reward in REWARDS, f"{reward} not in {REWARDS}"
    self.grid[x, y] = reward

  def add_terminal(self, x, y, terminal):
    assert terminal in TERMINALS, f"{terminal} not in {TERMINALS}"
    self.grid[x, y] = terminal

  def is_obstacle(self, x, y):
    if x < 0 or x >= self.width or y < 0 or y >= self.height:
      return True
    else:
      return self.grid[x ,y] in OBSTACLES

  def is_terminal(self, x, y):
    return self.grid[x ,y] in TERMINALS

  def get_reward(self, x, y):
    """
    Return the reward associated with a given location
    """
    return REWARDS[self.grid[x, y]]

  def get_next_state(self, current_state, action):
    """
    Get the next state given a current state and an action. The outcome can be
    stochastic  where rand_move_probability determines the probability of
    ignoring the action and performing a random move.
    """
    assert action in ACTIONS, f"Unknown acion {action} must be one of {ACTIONS}"

    x, y = current_state

    # If our current state is a terminal, there is no next state
    if self.grid[x, y] in TERMINALS:
      return None

    # Check of a random action should be performed:
    if np.random.rand() < rand_move_probability:
      action = np.random.choice(ACTIONS)

    if action == "up":      y -= 1
    elif action == "down":  y += 1
    elif action == "left":  x -= 1
    elif action == "right": x += 1

    # If the next state is an obstacle, stay in the current state
    return (x, y) if not self.is_obstacle(x, y) else current_state


## Basic example: Generating episodes

An episode is the series of states, actions and rewards reflecting an agent's experience interacting with the environment. An episode starts with an agent being placed at some initial state and continues till the agent reaches a terminal state.  To generate episodes, we first need a world and a policy:


In [4]:
world = World(2, 3)

# Since we only focus on episodic tasks, we must have a terminal state that the
# agent eventually reaches
world.add_terminal(1, 2, "+")

def equiprobable_random_policy(x, y):
  return { k:1/len(ACTIONS) for k in ACTIONS }

print(world.grid.T)

[[' ' ' ']
 [' ' ' ']
 [' ' '+']]


To generate an episode, we need to provide a ```World```, a policy, and a start state.

In each step, we do the following:
1. perform one of the actions (weighted random) returned by the policy for the giving state
2. get the reward and add a new entry to the episode $[S_t, A_t, R_{t+1}]$
3. move the agent to the next state

When a terminal state is reached, we return all the $[[S_0, A_0, R_1], ..., [S_{T}, A_T, R_{T+1}]]$ observed in the episode.

In [5]:
def generate_episode(world, policy, start_state):
    current_state = start_state
    episode = []
    while not world.is_terminal(*current_state):
        # Get the possible actions and their probabilities that our policy says
        # that the agent should perform in the current state:
        possible_actions = policy(*current_state)

        # Pick a weighted random action:
        action = random.choices(population=list(possible_actions.keys()),
                                weights=possible_actions.values(), k=1)

        # Get the next state from the world
        next_state = world.get_next_state(current_state, action[0])

        # Get the reward for performing the action
        reward = world.get_reward(*next_state)

        # Save the state, action and reward for this time step in our episode
        episode.append([current_state, action[0], reward])

        # Move the agent to the new state
        current_state = next_state

    return episode

Now, we can try to generate a couple of episodes and print the result:

In [6]:
for i in range(5):
    print(f"Episode {i}:")
    episode = generate_episode(world, equiprobable_random_policy, (0, 0))
    print(pd.DataFrame(episode, columns=["State", "Action", "Reward"]), end="\n\n")

Episode 0:
     State Action  Reward
0   (0, 0)     up       0
1   (0, 0)   left       0
2   (0, 0)  right       0
3   (1, 0)     up       0
4   (1, 0)  right       0
5   (1, 0)     up       0
6   (1, 0)   left       0
7   (0, 0)  right       0
8   (1, 0)   down       0
9   (1, 1)  right       0
10  (1, 1)   down      10

Episode 1:
    State Action  Reward
0  (0, 0)  right       0
1  (1, 0)   down       0
2  (1, 1)     up       0
3  (1, 0)  right       0
4  (1, 0)     up       0
5  (1, 0)  right       0
6  (1, 0)   down       0
7  (1, 1)   down      10

Episode 2:
    State Action  Reward
0  (0, 0)  right       0
1  (1, 0)  right       0
2  (1, 0)  right       0
3  (1, 0)   down       0
4  (1, 1)   left       0
5  (0, 1)     up       0
6  (0, 0)  right       0
7  (1, 0)   down       0
8  (1, 1)   down      10

Episode 3:
     State Action  Reward
0   (0, 0)   left       0
1   (0, 0)     up       0
2   (0, 0)   down       0
3   (0, 1)     up       0
4   (0, 0)   down       0
5   (0, 1)

### Exercise: Implement Monte Carlo-based prediction for state values

You should implement first-visit MC prediction for estimating $V≈v_\pi$. See page 92 of [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html).


In [7]:
### TODO: Implement your code here
def mc_first_pred(world, policy):
  V = np.full((world.width, world.height), 0.0)
  sum_of_returns = np.full((world.width, world.height), 0.0)
  times_visited = np.full((world.width, world.height), 0.0)

  for _ in range(10000):
    start_x = random.randint(0, world.width - 1)
    start_y = random.randint(0, world.height - 1)
    episode = generate_episode(world, policy, (start_x, start_y))

    G = 0
    #For loop through episodes backwards as int
    for i in range(len(episode)-1, -1, -1):
      G = gamma*G+episode[i][2]
      isVisited = False
      for j in range(0, i-1):
        if episode[i][0] == episode[j][0]:
          isVisited = True
          break
      if not isVisited:
        sum_of_returns[episode[i][0]] += G
        times_visited[episode[i][0]] += 1
        V[episode[i][0]] = sum_of_returns[episode[i][0]]/times_visited[episode[i][0]]

  return V



First, try your algorithm on the small $2\times3$ world above using an equiprobable policy and $\gamma = 0.9$. Depending on the number of episodes you use, you should get close to the true values:

<table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>3.283</td>
      <td>3.616</td>
    </tr>
    <tr>
      <th>1</th>
      <td>4.409</td>
      <td>5.556</td>
    </tr>
    <tr>
      <th>2</th>
      <td>6.349</td>
      <td>0.000</td>
    </tr>
  </tbody>
</table>




In [8]:
gamma = 0.9

### TODO: Implement your code here
v1 = mc_first_pred(world, equiprobable_random_policy)
display(pd.DataFrame(v1.T))

,0,1
0,3.291,3.625
1,4.437,5.540
2,6.304,0.000


Try to run your MC prediction code on worlds of different sizes (be careful not to make your world too large or you should have multiple terminals that an agent is likely to hit, otherwise it may take too long). You can try to change the policy as well, but rememeber that the agent **must** eventually reach a terminal state under any policy that you try.

In [14]:
### TODO: Implement your code here

world2 = World(10, 10)
world2.add_terminal(3, 3, "+")
world2.add_terminal(0, 0, "+")
v = mc_first_pred(world2, equiprobable_random_policy)
display(pd.DataFrame(v.T))

,0,1,2,3,4,5,6,7,8,9
0,0.000,4.948,2.639,1.745,1.185,0.720,0.436,0.268,0.149,0.111
1,5.102,3.504,2.537,2.246,1.552,0.922,0.533,0.304,0.156,0.112
2,2.750,2.585,3.065,4.354,2.455,1.240,0.638,0.310,0.163,0.108
3,1.778,2.292,4.193,0.000,3.920,1.596,0.702,0.323,0.165,0.111
4,1.224,1.553,2.471,3.996,2.303,1.158,0.577,0.288,0.143,0.101
5,0.729,0.887,1.272,1.659,1.195,0.695,0.380,0.212,0.123,0.088
6,0.440,0.495,0.609,0.675,0.559,0.380,0.237,0.145,0.082,0.066
7,0.231,0.262,0.292,0.303,0.268,0.207,0.133,0.091,0.063,0.048
8,0.138,0.162,0.169,0.153,0.137,0.117,0.091,0.063,0.042,0.032
9,0.104,0.109,0.118,0.109,0.089,0.085,0.062,0.046,0.031,0.027


### Exercise: Implement Monte Carlo-based prediction for state-action values

There is one more step that has to be in place before we can start to optimize a policy: estimating state-action values, $q_\pi(s,a)$, based on experience. Above where we estimated $v_\pi$, we only needed to keep track of the average return observed for _each state_. However, in order to estimate state-action values, we need to compute the average return observed for _each state-action_ pair.

That is, for every state $(0,0), (0,1), (0,2)...$ we need to compute different estimates for the four actions ```[ "up", "down", "left", "right" ]```

In [10]:
### TODO: Implement your code here to predict state-action values.

def mces(world):
  pol = np.full((world.width, world.height), None)
  q = np.full((world.width, world.height, 4), 0.0)
  sum_of_returns = np.full((world.width, world.height, 4), 0.0)
  times_visited = np.full((world.width, world.height, 4), 0.0)

  action_to_index = {action: i for i, action in enumerate(ACTIONS)}

  for _ in range(10000):
    start_x = random.randint(0, world.width - 1)
    start_y = random.randint(0, world.height - 1)
    episode = generate_episode(world, equiprobable_random_policy, (start_x, start_y))
    G = 0
    for i in range(len(episode)-1, -1, -1):
      G = gamma*G+episode[i][2]
      isVisited = False
      for j in range(0, i-1):
        if episode[i][0] == episode[j][0] and episode[i][1] == episode[j][1]:
          isVisited = True
          break
      action = action_to_index[episode[i][1]]
      if not isVisited:
        x,y = episode[i][0]
        sum_of_returns[x,y,action] += G
        times_visited[x,y,action] += 1
        q[x,y,action] = sum_of_returns[x,y,action]/times_visited[x,y,action]
        pol[x,y] = ACTIONS[np.argmax(q[x,y])]
  return pol




Try to experiment with your implementation by running it on different world sizes (be careful not to make your world too large or you should have multiple terminals that an agent is likely to hit, otherwise it may take too long), and try to experiment with different numbers of episodes:

In [11]:
### TODO: Implement your code here

pol = mces(world)
print(pol.T)

[['down' 'down']
 ['down' 'down']
 ['right' None]]


### Exercise: Implement on-policy Monte Carlo-based control with an $\epsilon$-soft policy

You are now ready to implement MC-based control (see page 101 of [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html) for the algorithm).

In your implementation, you need to update the state-action estimates like in the exercise above, but now, you also need implement an $ϵ$-soft policy that you can modify. How could you do that?

_Hint_: You can either represent your policy explicitly. That is, for each state $(x,y)$ you have a ```dict``` with actions and their probabilities which you then update each time you step through an episode. When the policy is called, it then just returns the ```dict``` with action probablities corresponding to the current state.

Alternatively, you can compute the action probabilities when your policy is called based on the current action-values estimates.

In [25]:
gamma = 0.9
epsilon = 0.1

### TODO: Implement you code here. You need to define a policy function
###       and then the actual algorithm that goes through time step in each
###       episode and updates state-action values and the policy. Also, make
###       sure that you can print out the policy learned, that is, the action
###       with the highest expected value in each state.


### TODO: Implement your code here to predict state-action values.


sum_of_returns_soft = np.full((world.width, world.height, 4), 0.0)
times_visited_soft = np.full((world.width, world.height, 4), 1.0)

def mc_soft_policy(x,y):

  Q = []
  for a in ACTIONS:
    Q.append(sum_of_returns_soft[x,y,ACTIONS.index(a)] / times_visited_soft[x,y,ACTIONS.index(a)])

  best_action = ACTIONS[np.argmax(Q)]
  if(np.random.rand() > epsilon ):
    return {best_action:1}
  else:
    return {k:1/len(ACTIONS) for k in ACTIONS}






def mc_control(world):
  pol = np.full((world.width, world.height), None)
  q = np.full((world.width, world.height, 4), 0.0)
  sum_of_returns = np.full((world.width, world.height, 4), 0.0)
  times_visited = np.full((world.width, world.height, 4), 0.0)

  action_to_index = {action: i for i, action in enumerate(ACTIONS)}

  for _ in range(10000):
    start_x = random.randint(0, world.width - 1)
    start_y = random.randint(0, world.height - 1)
    episode = generate_episode(world, mc_soft_policy, (start_x, start_y))
    G = 0
    for i in range(len(episode)-1, -1, -1):
      G = gamma*G+episode[i][2]
      isVisited = False
      for j in range(0, i-1):
        if episode[i][0] == episode[j][0] and episode[i][1] == episode[j][1]:
          isVisited = True
          break
      action = action_to_index[episode[i][1]]
      if not isVisited:
        x,y = episode[i][0]
        sum_of_returns[x,y,action] += G
        times_visited[x,y,action] += 1
        q[x,y,action] = sum_of_returns[x,y,action]/times_visited[x,y,action]
        pol[x,y] = ACTIONS[np.argmax(q[x,y])]
  return pol



world3 = World(10, 10)
world3.add_terminal(3, 3, "+")
world3.add_terminal(0, 0, "+")
nypol = mc_control(world3)
print(nypol.T)


IndexError: index 2 is out of bounds for axis 0 with size 2

Try to experiment with your implementation by running it on different world sizes (be careful not to make your world too large or you should have multiple terminals that an agent is likely to hit, otherwise it may take too long), try to experiment with different numbers of episodes, and different values of epsilon:

In [21]:
### TODO: Implement your code here


IndexError: index 4 is out of bounds for axis 0 with size 2

### Optional exercise

Try to implement exploring starts (see page 99 of [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html) for the algorithm). It should be straightforward and only require minimal changes to the code for the exercise above.